In [ ]:
import sys
import os
sys.path.append('../')
import pathlib as pl
from SymEigen import *
from sympy import symbols
from project_dir import project_dir

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")


In [ ]:
kappa = Eigen.Scalar("kappa")
v = Eigen.Scalar("v")
q = Eigen.Vector("q", 12)
q_a = Matrix(q[3:12])

Cl = Gen.Closure(kappa, v, q)

q_a

In [ ]:
a = [Matrix(q_a[0:3]), Matrix(q_a[3:6]), Matrix(q_a[6:9])]
a

In [ ]:
E = 0
for i in range(3):
    E+=(a[i].dot(a[i]) - 1)**2
for i in range(3):
    for j in range(3):
        if i!=j:
            E+=(a[i].dot(a[j]))**2

E*=kappa * v

E

In [ ]:
dEdq = VecDiff(E,q_a)
dEdq

In [ ]:
ddEddq = VecDiff(dEdq, q_a)
ddEddq

In [ ]:
s = f'''// Affine body Orthogonal Potential

{Cl("E",E)}
{Cl("dEdq",dEdq)}
{Cl("ddEddq",ddEddq)}
'''
print(s)

f = open( project_dir() / 'src/uipc/backends/cuda/affine_body/constitutions/sym/ortho_potential.inl', 'w')
f.write(s)
f.close()